### Various Dev-Testing, Idea Testing, Brainstorm, Logic-dev Notebook for RAG.

### ISOLATION TESTS - Test Year, Company, Metric, Warning, Section, Risk - Extraction !

In [1]:
from pathlib import Path
import sys

# in: notebooks-experiments/

# Step 1: Go UP to project root
project_root = Path.cwd().parent  # Up 1 level: finrag_ml_tg1/

# Step 2: Verify (so you KNOW it's right)
print(f"Current dir: {Path.cwd()}")
print(f"Project root: {project_root}")
print(f"rag_modules_src exists? {(project_root / 'rag_modules_src').exists()}")

# Step 3: Add to path
sys.path.insert(0, str(project_root))

# Step 4: Import
import rag_modules_src
from rag_modules_src.entity_adapter import company_extractor

Current dir: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\notebooks-experiments
Project root: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1
rag_modules_src exists? True


In [2]:
## Company testing for entity_adapter/company_universe.py and extractor, etc.

import logging

logging.basicConfig(
    level=logging.INFO,  
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,
)

import logging

logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("rag_modules_src.entity_adapter").setLevel(logging.INFO)
logging.getLogger("rag_modules_src.entity_adapter.company_extractor").setLevel(logging.INFO)
logging.getLogger("rag_modules_src.entity_adapter.company_universe").setLevel(logging.INFO)


## confusing: basicConfig sees handlers and silently returns without changing anything.
## Root logger stays at DEBUG, and all child loggers inherit that.
# First basicConfig(level=DEBUG) “wins the config”.
# Later basicConfig(level=INFO) is a no-op because logging is already configured.


# ---
import sys
from pathlib import Path

from rag_modules_src.entity_adapter.company_universe import CompanyUniverse
from rag_modules_src.entity_adapter.company_extractor import CompanyExtractor


project_root = Path.cwd().parent  # Up to finrag_ml_tg1/

file_name = "finrag_dim_companies_21.parquet"
dim_path = project_root / "data_cache" / "dimensions" / file_name

universe = CompanyUniverse(dim_path=dim_path)
extractor = CompanyExtractor(universe)


queries = [
    # 1. Mixed tickers, brand names, possessives
    "Between NVDA, Apple's services business, and Microsoft’s cloud, who grew revenue fastest in 2023?",

    # 2. Misspellings / fuzzy alias for names
    "How did Microsft and Teslsa talk about AI and autonomy in 2022 filings?",
    "How did Microsft's and Teslsa's and Apple's and Walmart's talk about AI and autonomy in 2022 filings?",

    # 3. Full legal names + brand names mixed
    "Compare EXXON MOBIL CORP, Netflix, and Meta Platforms on operating cash flow in 2021.",

    # 4. CIK-only plus casual brand mention (universe should pick both)
    "For CIK 0000320193 and CIK 0000789019, and also just 'Nvidia' for reference, show net income in 2020.",

    # 5. Lowercase + punctuation + plural / possessive noise
    "do apple, amazon, and walmart's disclosures say anything about supply chain risks in 2021?",

    # 6. Potential confusion with common word vs ticker (COST)
    "Did Costco (ticker COST) and Walmart change their dividend policy after 2020?",

    # 7. Repeated + mixed forms in one sentence
    "NVIDIA, nvda, Nvidia's management and META, Meta Platforms, Inc. all discussed AI – extract all companies mentioned.",

    # 8. Noise, extra numbers, and inline comments
    "In 2019, 2020, and 2023, how did 'Genworth Financial Inc' and 'Radian Group' (CIK 1276520 and 0000890926) describe mortgage insurance trends?",
    "In 2019, 2020, and 2023, how did 'Genworth Financial Inc' and 'Radian Group' describe mortgage insurance trends?",
]


for q in queries:
    matches = extractor.extract(q)
    print("Query:", q)
    print("  CIKs int:   ", matches.ciks_int)
    print("  CIKs str:  ", matches.ciks_str)
    print("  Tickers:", matches.tickers)
    print("  Names:  ", matches.names)
    print()


2025-11-16 20:36:46,232 - rag_modules_src.entity_adapter.company_universe - INFO - Loading company dim from: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_companies_21.parquet
2025-11-16 20:36:46,325 - rag_modules_src.entity_adapter.company_universe - INFO - Loaded dim with 21 rows and columns: ['company_id', 'cik_int', 'cik', 'company_name', 'ticker', 'source', 'tier', 'quality_score', 'selection_source', 'rank_within_group', 'selected_at', 'version']
2025-11-16 20:36:46,328 - rag_modules_src.entity_adapter.company_universe - INFO - Building indexes from dim with 21 valid rows
2025-11-16 20:36:46,333 - rag_modules_src.entity_adapter.company_universe - INFO - CompanyUniverse initialized: 21 companies, 21 tickers, 21 alias tokens
2025-11-16 20:36:46,334 - rag_modules_src.entity_adapter.company_extractor - INFO - Extracting companies from query: "Between NVDA, Apple's services business, and Microsoft’s clo

Query: Between NVDA, Apple's services business, and Microsoft’s cloud, who grew revenue fastest in 2023?
  CIKs int:    [320193, 789019, 1045810]
  CIKs str:   ['0000320193', '0000789019', '0001045810']
  Tickers: ['AAPL', 'MSFT', 'NVDA']
  Names:   ['Apple Inc.', 'MICROSOFT CORP', 'NVIDIA CORP']

Query: How did Microsft and Teslsa talk about AI and autonomy in 2022 filings?
  CIKs int:    [789019]
  CIKs str:   ['0000789019']
  Tickers: ['MSFT']
  Names:   ['MICROSOFT CORP']

Query: How did Microsft's and Teslsa's and Apple's and Walmart's talk about AI and autonomy in 2022 filings?
  CIKs int:    [104169, 320193, 789019]
  CIKs str:   ['0000104169', '0000320193', '0000789019']
  Tickers: ['AAPL', 'MSFT', 'WMT']
  Names:   ['Apple Inc.', 'MICROSOFT CORP', 'Walmart Inc.']

Query: Compare EXXON MOBIL CORP, Netflix, and Meta Platforms on operating cash flow in 2021.
  CIKs int:    [34088, 1065280, 1326801]
  CIKs str:   ['0000034088', '0001065280', '0001326801']
  Tickers: ['META', 'NFLX

In [3]:
## testing - year extractor AND ALSO company extractor together.

import logging
from pathlib import Path

from rag_modules_src.entity_adapter.company_universe import CompanyUniverse
from rag_modules_src.entity_adapter.company_extractor import CompanyExtractor
from rag_modules_src.entity_adapter.year_extractor import YearExtractor

logging.basicConfig(
    level=logging.WARNING,  
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,
)

project_root = Path.cwd().parent
file_name = "finrag_dim_companies_21.parquet"
dim_path = project_root / "data_cache" / "dimensions" / file_name

universe = CompanyUniverse(dim_path=dim_path)
company_extractor = CompanyExtractor(universe)
year_extractor = YearExtractor()

queries = [
    "In 2019, 2020, and 2023, how did 'Genworth Financial Inc' and 'Radian Group' (CIK 1276520 and 0000890926) describe mortgage insurance trends?",
    "In 2019, 2021, and 2025, how did 'Genworth Financial Inc' and 'Radian Group' describe mortgage insurance trends?",
    "What was NVDA's revenue in 2023?",
    "What was Nvidia's, Apple's and Amazon's revenue in 2021, 2022, and 2023?",
    "Compare Amazon, Apple and Microsoft performance in 2022",
    "How did NVIDIA describe its liquidity risk in 2020? (CIK 0001045810)",
    "Compare NVDA and Apple in 2023",
    "Show cash from operations for Meta and Tesla in 2024 and 2026",
]

for q in queries:
    c_matches = company_extractor.extract(q)
    y_matches = year_extractor.extract(q)

    print("Query:", q)
    print("  CIKs int:   ", c_matches.ciks_int)
    print("  CIKs str:   ", c_matches.ciks_str)
    print("  Tickers:    ", c_matches.tickers)
    print("  Names:      ", c_matches.names)
    
    print("  Years:  ", y_matches.years)
    print("    past:     ", y_matches.past_years)
    print("    current:  ", y_matches.current_years)
    print("    future:   ", y_matches.future_years)
    print("  Warning:    ", y_matches.warning)
    print()





2025-11-16 20:36:46,538 - rag_modules_src.entity_adapter.company_universe - INFO - Loading company dim from: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_companies_21.parquet
2025-11-16 20:36:46,541 - rag_modules_src.entity_adapter.company_universe - INFO - Loaded dim with 21 rows and columns: ['company_id', 'cik_int', 'cik', 'company_name', 'ticker', 'source', 'tier', 'quality_score', 'selection_source', 'rank_within_group', 'selected_at', 'version']
2025-11-16 20:36:46,543 - rag_modules_src.entity_adapter.company_universe - INFO - Building indexes from dim with 21 valid rows
2025-11-16 20:36:46,545 - rag_modules_src.entity_adapter.company_universe - INFO - CompanyUniverse initialized: 21 companies, 21 tickers, 21 alias tokens
2025-11-16 20:36:46,547 - rag_modules_src.entity_adapter.company_extractor - INFO - Extracting companies from query: "In 2019, 2020, and 2023, how did 'Genworth Financial Inc' an

Query: In 2019, 2020, and 2023, how did 'Genworth Financial Inc' and 'Radian Group' (CIK 1276520 and 0000890926) describe mortgage insurance trends?
  CIKs int:    [890926, 1276520]
  CIKs str:    ['0000890926', '0001276520']
  Tickers:     ['GNW', 'RDN']
  Names:       ['GENWORTH FINANCIAL INC', 'RADIAN GROUP INC']
  Years:   [2019, 2020, 2023]
    past:      [2019, 2020, 2023]
    current:   []
    future:    []

Query: In 2019, 2021, and 2025, how did 'Genworth Financial Inc' and 'Radian Group' describe mortgage insurance trends?
  CIKs int:    [890926, 1276520]
  CIKs str:    ['0000890926', '0001276520']
  Tickers:     ['GNW', 'RDN']
  Names:       ['GENWORTH FINANCIAL INC', 'RADIAN GROUP INC']
  Years:   [2019, 2021, 2025]
    past:      [2019, 2021]
    current:   [2025]
    future:    []

Query: What was NVDA's revenue in 2023?
  CIKs int:    [1045810]
  CIKs str:    ['0001045810']
  Tickers:     ['NVDA']
  Names:       ['NVIDIA CORP']
  Years:   [2023]
    past:      [2023]
   

In [4]:
import logging
from pathlib import Path

logging.basicConfig(
    level=logging.INFO,  
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,
)

from rag_modules_src.entity_adapter.company_universe import CompanyUniverse
from rag_modules_src.entity_adapter.company_extractor import CompanyExtractor
from rag_modules_src.entity_adapter.year_extractor import YearExtractor
from rag_modules_src.entity_adapter.metric_adapter import MetricAdapter

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,
)

project_root = Path.cwd().parent
dim_path = project_root / "data_cache" / "dimensions" / "finrag_dim_companies_21.parquet"

universe = CompanyUniverse(dim_path=dim_path)
company_extractor = CompanyExtractor(universe)
year_extractor = YearExtractor()
metric_adapter = MetricAdapter()

queries = [
    "What was NVDA's revenue in 2023?",
    "What was Nvidia's, Apple's and Amazon's revenue and net income in 2021, 2022, and 2023?",
    "Compare Amazon, Apple and Microsoft operating cash flow and gross margin in 2022",
    "How did NVIDIA describe its liquidity risk and operating expenses in 2020? (CIK 0001045810)",
]

for q in queries:
    c = company_extractor.extract(q)
    y = year_extractor.extract(q)
    m = metric_adapter.extract(q)

    print("Query:", q)
    print("  CIKs int:   ", c.ciks_int)
    print("  CIKs str:   ", c.ciks_str)
    print("  Tickers:    ", c.tickers)
    print("  Names:      ", c.names)
    print("  Years:      ", y.years)
    print("  Metrics:    ", m.metrics)
    print()


2025-11-16 20:36:46,844 - rag_modules_src.entity_adapter.company_universe - INFO - Loading company dim from: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_companies_21.parquet
2025-11-16 20:36:46,847 - rag_modules_src.entity_adapter.company_universe - INFO - Loaded dim with 21 rows and columns: ['company_id', 'cik_int', 'cik', 'company_name', 'ticker', 'source', 'tier', 'quality_score', 'selection_source', 'rank_within_group', 'selected_at', 'version']
2025-11-16 20:36:46,849 - rag_modules_src.entity_adapter.company_universe - INFO - Building indexes from dim with 21 valid rows
2025-11-16 20:36:46,852 - rag_modules_src.entity_adapter.company_universe - INFO - CompanyUniverse initialized: 21 companies, 21 tickers, 21 alias tokens
2025-11-16 20:36:46,853 - rag_modules_src.entity_adapter.metric_adapter - INFO - FilterExtractorV2 initialized with 94 metric mapping entries from metric_mapping_v2
2025-11-16 20

Query: What was NVDA's revenue in 2023?
  CIKs int:    [1045810]
  CIKs str:    ['0001045810']
  Tickers:     ['NVDA']
  Names:       ['NVIDIA CORP']
  Years:       [2023]
  Metrics:     ['income_stmt_Revenue']

Query: What was Nvidia's, Apple's and Amazon's revenue and net income in 2021, 2022, and 2023?
  CIKs int:    [320193, 1018724, 1045810]
  CIKs str:    ['0000320193', '0001018724', '0001045810']
  Tickers:     ['AAPL', 'AMZN', 'NVDA']
  Names:       ['AMAZON COM INC', 'Apple Inc.', 'NVIDIA CORP']
  Years:       [2021, 2022, 2023]
  Metrics:     ['income_stmt_Net Income', 'income_stmt_Revenue']

Query: Compare Amazon, Apple and Microsoft operating cash flow and gross margin in 2022
  CIKs int:    [320193, 789019, 1018724]
  CIKs str:    ['0000320193', '0000789019', '0001018724']
  Tickers:     ['AAPL', 'AMZN', 'MSFT']
  Names:       ['AMAZON COM INC', 'Apple Inc.', 'MICROSOFT CORP']
  Years:       [2022]
  Metrics:     ['Gross Profit Margin %', 'cash_flow_Operating Cash Flow']



In [5]:
queries = [
    # existing ones...
    "What was Nvidia's, Apple's and Amazon's revenue and net income in 2021, 2022, and 2023?",

    # Wild test 1
    "Between Nvidia's gaming business, Apple's services, Teslsa, and MSFT, "
    "compare revenue, net income, operating cash flow, total assets and gross "
    "profit margin for 2019, 2020, 2023 and 2026.",

    # Wild test 2
    "For EXXON MOBIL CORP, Netflix's streaming business, META, and Walmrt, "
    "show operating expenses, interest expense, cash flow from operations, "
    "total liabilities, shareholders' equity and earnings in 2018, 2020, 2022 and 2030.",

    "What risks do Amazon's and Apple's in its Risk Factors sections across years 1990-1994, 1800 - 1820, 2015 to 2020?",
]

for q in queries:
    c = company_extractor.extract(q)
    y = year_extractor.extract(q)
    m = metric_adapter.extract(q)

    print("Query:", q)
    print("  CIKs int:   ", c.ciks_int)
    print("  CIKs str:   ", c.ciks_str)
    print("  Tickers:    ", c.tickers)
    print("  Names:      ", c.names)
    print("  Years:      ", y.years)
    print("  Metrics:    ", m.metrics)
    print("  Warning:    ", y.warning)
    print()


2025-11-16 20:36:47,125 - rag_modules_src.entity_adapter.company_extractor - INFO - Extracting companies from query: "What was Nvidia's, Apple's and Amazon's revenue and net income in 2021, 2022, and 2023?"
2025-11-16 20:36:47,127 - rag_modules_src.entity_adapter.company_extractor - INFO - Extraction result: ciks_int=[320193, 1018724, 1045810], tickers=['AAPL', 'AMZN', 'NVDA'], names=['AMAZON COM INC', 'Apple Inc.', 'NVIDIA CORP']
2025-11-16 20:36:47,128 - rag_modules_src.entity_adapter.metric_adapter - INFO - MetricAdapter extracting metrics from query: "What was Nvidia's, Apple's and Amazon's revenue and net income in 2021, 2022, and 2023?"
2025-11-16 20:36:47,141 - rag_modules_src.entity_adapter.metric_adapter - INFO - MetricAdapter found metrics: ['income_stmt_Net Income', 'income_stmt_Revenue']
2025-11-16 20:36:47,142 - rag_modules_src.entity_adapter.company_extractor - INFO - Extracting companies from query: "Between Nvidia's gaming business, Apple's services, Teslsa, and MSFT, c

Query: What was Nvidia's, Apple's and Amazon's revenue and net income in 2021, 2022, and 2023?
  CIKs int:    [320193, 1018724, 1045810]
  CIKs str:    ['0000320193', '0001018724', '0001045810']
  Tickers:     ['AAPL', 'AMZN', 'NVDA']
  Names:       ['AMAZON COM INC', 'Apple Inc.', 'NVIDIA CORP']
  Years:       [2021, 2022, 2023]
  Metrics:     ['income_stmt_Net Income', 'income_stmt_Revenue']

Query: Between Nvidia's gaming business, Apple's services, Teslsa, and MSFT, compare revenue, net income, operating cash flow, total assets and gross profit margin for 2019, 2020, 2023 and 2026.
  CIKs int:    [320193, 789019, 1045810]
  CIKs str:    ['0000320193', '0000789019', '0001045810']
  Tickers:     ['AAPL', 'MSFT', 'NVDA']
  Names:       ['Apple Inc.', 'MICROSOFT CORP', 'NVIDIA CORP']
  Years:       [2019, 2020, 2023, 2026]
  Metrics:     ['Gross Profit Margin %', 'balance_sheet_Total Assets', 'cash_flow_Operating Cash Flow', 'income_stmt_Net Income', 'income_stmt_Revenue']

Query: For 

In [6]:
## Full entity + section + risk testing cell

import logging
from pathlib import Path

from rag_modules_src.entity_adapter.company_universe import CompanyUniverse
from rag_modules_src.entity_adapter.company_extractor import CompanyExtractor
from rag_modules_src.entity_adapter.year_extractor import YearExtractor
from rag_modules_src.entity_adapter.metric_adapter import MetricAdapter
from rag_modules_src.entity_adapter.section_universe import SectionUniverse
from rag_modules_src.entity_adapter.section_extractor import SectionExtractor

# Keep logs to INFO for normal testing
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,
)

# ---------------------------------------------------------------------
# Set up universes / adapters
# ---------------------------------------------------------------------

project_root = Path.cwd().parent  # up to finrag_ml_tg1/

# Company universe
dim_companies_path = project_root / "data_cache" / "dimensions" / "finrag_dim_companies_21.parquet"
company_universe = CompanyUniverse(dim_path=dim_companies_path)
company_extractor = CompanyExtractor(company_universe)

# Year extractor
year_extractor = YearExtractor()

# Metric adapter (reusing old metric pipeline + v2 mappings)
metric_adapter = MetricAdapter()

# Section universe + extractor
dim_sections_path = project_root / "data_cache" / "dimensions" / "finrag_dim_sec_sections.parquet"
section_universe = SectionUniverse(dim_path=dim_sections_path)
section_extractor = SectionExtractor(section_universe)

# ---------------------------------------------------------------------
# Test queries (extended)
# ---------------------------------------------------------------------

queries = [
    # Existing core multi-entity test
    "What was Nvidia's, Apple's and Amazon's revenue and net income in 2021, 2022, and 2023?",

    # Wild test 1: fuzzy names, tickers, lots of metrics, mix of years incl. future
    "Between Nvidia's gaming business, Apple's services, Teslsa, and MSFT, "
    "compare revenue, net income, operating cash flow, total assets and gross "
    "profit margin for 2019, 2020, 2023 and 2026.",

    # Wild test 2: multiple companies (legal names + brands), lots of metrics, future year
    "For EXXON MOBIL CORP, Netflix's streaming business, META, and Walmrt, "
    "show operating expenses, interest expense, cash flow from operations, "
    "total liabilities, shareholders' equity and earnings in 2018, 2020, 2022 and 2030.",

    # Risk factors + year ranges
    "What risks do Amazon's and Apple's in its Risk Factors sections across years "
    "1990-1994, 1800 - 1820, 2015 to 2020?",

    # NEW: explicit item references + MD&A language
    "In the MD&A (Item 7), how did NVIDIA discuss liquidity and capital resources "
    "and operating results in 2020?",

    # NEW: risk + market risk items + risk-topic cues
    "Looking at Item 1A and Item 7A, what liquidity and market risk factors did "
    "Tesla and Microsoft highlight around foreign currency risk and refinancing "
    "in 2019 and 2021?",

    # Mad Stress-Test Query
    "In Item 1, Item 1A, and also kinda Item-7 and item_8, what did Nvidia’s gaming division, Apple Inc.’s "
    "services, Amazn, Microsft, EXXON MOBIL CORP, Meta Platforms, and Walmrt disclose about revenue, net income, " 
    "gross profit margin, operating cash flow, total liabilities, shareholders’ equity, interest expense, and COGS during "
    "the periods 1990–1994, 2001 to 2003, 2015-2020, 2023, 2026, and 2035, especially regarding liquidity risks, market risk, " 
    "operational risks, and any MD&A commentary on outlook and results of operations? "
]

# ---------------------------------------------------------------------
# Run extraction pipeline
# ---------------------------------------------------------------------

for q in queries:
    c = company_extractor.extract(q)
    y = year_extractor.extract(q)
    m = metric_adapter.extract(q)
    s = section_extractor.extract_sections(q)
    r = section_extractor.extract_risk_topics(q)

    print("Query:", q)
    print("  CIKs int:    ", c.ciks_int)
    print("  CIKs str:    ", c.ciks_str)
    print("  Tickers:     ", c.tickers)
    print("  Names:       ", c.names)
    print("  Years:       ", y.years)
    print("  Metrics:     ", m.metrics)
    print("  Sections:    ", s.items, "  primary:", s.primary)
    print("  Risk topics: ", r.topics)
    print("  Warning:     ", y.warning)
    print("-" * 120)
    print()


2025-11-16 20:36:47,554 - rag_modules_src.entity_adapter.company_universe - INFO - Loading company dim from: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_companies_21.parquet
2025-11-16 20:36:47,557 - rag_modules_src.entity_adapter.company_universe - INFO - Loaded dim with 21 rows and columns: ['company_id', 'cik_int', 'cik', 'company_name', 'ticker', 'source', 'tier', 'quality_score', 'selection_source', 'rank_within_group', 'selected_at', 'version']
2025-11-16 20:36:47,559 - rag_modules_src.entity_adapter.company_universe - INFO - Building indexes from dim with 21 valid rows
2025-11-16 20:36:47,561 - rag_modules_src.entity_adapter.company_universe - INFO - CompanyUniverse initialized: 21 companies, 21 tickers, 21 alias tokens
2025-11-16 20:36:47,562 - rag_modules_src.entity_adapter.metric_adapter - INFO - FilterExtractorV2 initialized with 94 metric mapping entries from metric_mapping_v2
2025-11-16 20

Query: What was Nvidia's, Apple's and Amazon's revenue and net income in 2021, 2022, and 2023?
  CIKs int:     [320193, 1018724, 1045810]
  CIKs str:     ['0000320193', '0001018724', '0001045810']
  Tickers:      ['AAPL', 'AMZN', 'NVDA']
  Names:        ['AMAZON COM INC', 'Apple Inc.', 'NVIDIA CORP']
  Years:        [2021, 2022, 2023]
  Metrics:      ['income_stmt_Net Income', 'income_stmt_Revenue']
  Sections:     []   primary: None
  Risk topics:  []
------------------------------------------------------------------------------------------------------------------------

Query: Between Nvidia's gaming business, Apple's services, Teslsa, and MSFT, compare revenue, net income, operating cash flow, total assets and gross profit margin for 2019, 2020, 2023 and 2026.
  CIKs int:     [320193, 789019, 1045810]
  CIKs str:     ['0000320193', '0000789019', '0001045810']
  Tickers:      ['AAPL', 'MSFT', 'NVDA']
  Names:        ['Apple Inc.', 'MICROSOFT CORP', 'NVIDIA CORP']
  Years:        [201

2025-11-16 20:36:47,769 - rag_modules_src.entity_adapter.company_extractor - INFO - Extracting companies from query: 'In Item 1, Item 1A, and also kinda Item-7 and item_8, what did Nvidia’s gaming division, Apple Inc.’s services, Amazn, Microsft, EXXON MOBIL CORP, Meta Platforms, and Walmrt disclose about revenue, net income, gross profit margin, operating cash flow, total liabilities, shareholders’ equity, interest expense, and COGS during the periods 1990–1994, 2001 to 2003, 2015-2020, 2023, 2026, and 2035, especially regarding liquidity risks, market risk, operational risks, and any MD&A commentary on outlook and results of operations? '
2025-11-16 20:36:47,775 - rag_modules_src.entity_adapter.company_extractor - INFO - Extraction result: ciks_int=[34088, 104169, 320193, 789019, 1045810, 1326801], tickers=['AAPL', 'META', 'MSFT', 'NVDA', 'WMT', 'XOM'], names=['Apple Inc.', 'EXXON MOBIL CORP', 'MICROSOFT CORP', 'Meta Platforms, Inc.', 'NVIDIA CORP', 'Walmart Inc.']
2025-11-16 20:36:4

Query: Looking at Item 1A and Item 7A, what liquidity and market risk factors did Tesla and Microsoft highlight around foreign currency risk and refinancing in 2019 and 2021?
  CIKs int:     [789019, 1318605]
  CIKs str:     ['0000789019', '0001318605']
  Tickers:      ['MSFT', 'TSLA']
  Names:        ['MICROSOFT CORP', 'Tesla, Inc.']
  Years:        [2019, 2021]
  Metrics:      []
  Sections:     ['ITEM_1A', 'ITEM_7', 'ITEM_7A']   primary: ITEM_7
  Risk topics:  ['liquidity_credit', 'general_risk']
------------------------------------------------------------------------------------------------------------------------

Query: In Item 1, Item 1A, and also kinda Item-7 and item_8, what did Nvidia’s gaming division, Apple Inc.’s services, Amazn, Microsft, EXXON MOBIL CORP, Meta Platforms, and Walmrt disclose about revenue, net income, gross profit margin, operating cash flow, total liabilities, shareholders’ equity, interest expense, and COGS during the periods 1990–1994, 2001 to 2003, 20

In [7]:
from pathlib import Path
from rag_modules_src.entity_adapter.entity_adapter import EntityAdapter


adapter = EntityAdapter()
print("project_root:", adapter.project_root)
print("company_dim:", adapter.company_universe.dim_path)
print("sections_dim:", adapter.section_universe.dim_path)

queries = [
    "What was Nvidia's, Apple's and Amazon's revenue and net income in 2021, 2022, and 2023?",
    "Between Nvidia's gaming business, Apple's services, Teslsa, and MSFT, "
    "compare revenue, net income, operating cash flow, total assets and gross "
    "profit margin for 2019, 2020, 2023 and 2026.",
    
    "In Item 1A and Item 7, what liquidity and market risks did Tesla highlight in 2020?",
]

for q in queries:
    res = adapter.extract(q)
    EntityAdapter.debug_print(res)


2025-11-16 20:36:48,310 - rag_modules_src.entity_adapter.company_universe - INFO - Loading company dim from: D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_companies_21.parquet
2025-11-16 20:36:48,313 - rag_modules_src.entity_adapter.company_universe - INFO - Loaded dim with 21 rows and columns: ['company_id', 'cik_int', 'cik', 'company_name', 'ticker', 'source', 'tier', 'quality_score', 'selection_source', 'rank_within_group', 'selected_at', 'version']
2025-11-16 20:36:48,315 - rag_modules_src.entity_adapter.company_universe - INFO - Building indexes from dim with 21 valid rows
2025-11-16 20:36:48,317 - rag_modules_src.entity_adapter.company_universe - INFO - CompanyUniverse initialized: 21 companies, 21 tickers, 21 alias tokens
2025-11-16 20:36:48,321 - rag_modules_src.entity_adapter.section_universe - INFO - SectionUniverse loaded 21 sections from D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Proje

project_root: D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1
company_dim: D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_companies_21.parquet
sections_dim: D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_sec_sections.parquet
Query: What was Nvidia's, Apple's and Amazon's revenue and net income in 2021, 2022, and 2023?
  CIKs int:      [320193, 1018724, 1045810]
  CIKs str:      ['0000320193', '0001018724', '0001045810']
  Tickers:       ['AAPL', 'AMZN', 'NVDA']
  Names:         ['AMAZON COM INC', 'Apple Inc.', 'NVIDIA CORP']
  Years:         [2021, 2022, 2023]
  Metrics:       ['income_stmt_Net Income', 'income_stmt_Revenue']
  Sections:      []
  Primary sec:   None
  Risk topics:   []
  Year warning:  None

Query: Between Nvidia's gaming business, Apple's services, Teslsa, an

In [8]:
from pathlib import Path
from rag_modules_src.entity_adapter.entity_adapter import EntityAdapter

logging.disable(logging.CRITICAL)  

# (optional – you can drop this entirely if you don't need formatting)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,
)


adapter = EntityAdapter()
print("project_root:", adapter.project_root)
print("company_dim:", adapter.company_universe.dim_path)
print("sections_dim:", adapter.section_universe.dim_path)

queries = [
    "What was Nvidia's, Apple's and Amazon's revenue and net income in 2021, 2022, and 2023?",
    "Between Nvidia's gaming business, Apple's services, Teslsa, and MSFT, "
    "compare revenue, net income, operating cash flow, total assets and gross "
    "profit margin for 2019, 2020, 2023 and 2026.",
    
    "In Item 1A and Item 7, what liquidity and market risks did Tesla highlight in 2020?",
]

for q in queries:
    res = adapter.extract(q)

## print the shape of res.

res = adapter.extract(q)
print(res.__dict__.keys())  # dict_keys([...])


project_root: D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1
company_dim: D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_companies_21.parquet
sections_dim: D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_sec_sections.parquet
dict_keys(['query', 'companies', 'years', 'metrics', 'sections', 'primary_section', 'risk_topics'])


### Isolation test - for query embed.
- finds FinSights by walking parents.
- sets project_root = FinSights/ModelPipeline/finrag_ml_tg1.
- adds that to sys.path.
- 🔹 Cell 1 — Imports, sys.path, config, adapter + embedder
- 🔹 Cell 2 — Real in-scope smoke test (Bedrock call)
- 🔹 Cell 3 — Guardrail tests (no Bedrock cost on failures)

In [1]:
from pathlib import Path
import sys, logging, yaml

# Find FinSights root, navigate to project
current = Path.cwd()
for parent in [current] + list(current.parents):
    if parent.name == "FinSights":
        finsights_root = parent
        break
else:
    raise RuntimeError("Cannot find FinSights root")

project_root = finsights_root / "ModelPipeline" / "finrag_ml_tg1"
assert project_root.exists(), f"Project not found: {project_root}"

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✓ Project root: {project_root}")

from rag_modules_src.entity_adapter.entity_adapter import EntityAdapter
from rag_modules_src.utilities.query_embedder_v2 import (
    EmbeddingRuntimeConfig,
    QueryEmbedderV2,
    QueryTooLongError,
    QueryTooShortError,
    QueryOutOfScopeError,
)

logging.basicConfig(level=logging.INFO)

config_path = project_root / ".aws_config" / "ml_config.yaml"
assert config_path.exists(), f"Config not found: {config_path}"

cfg_raw = yaml.safe_load(config_path.read_text())
embedding_cfg_dict = cfg_raw["embedding"]
runtime_cfg = EmbeddingRuntimeConfig.from_ml_config(embedding_cfg_dict)
runtime_cfg.max_query_chars = 1500

print("EmbeddingRuntimeConfig loaded")

adapter = EntityAdapter()
embedder = QueryEmbedderV2(runtime_cfg)
print("✓ All components initialized")

## ==================================================================



✓ Project root: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1


INFO:rag_modules_src.entity_adapter.company_universe:Loading company dim from: D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_companies_21.parquet
INFO:rag_modules_src.entity_adapter.company_universe:Loaded dim with 21 rows and columns: ['company_id', 'cik_int', 'cik', 'company_name', 'ticker', 'source', 'tier', 'quality_score', 'selection_source', 'rank_within_group', 'selected_at', 'version']
INFO:rag_modules_src.entity_adapter.company_universe:Building indexes from dim with 21 valid rows
INFO:rag_modules_src.entity_adapter.company_universe:CompanyUniverse initialized: 21 companies, 21 tickers, 21 alias tokens
INFO:rag_modules_src.entity_adapter.section_universe:SectionUniverse loaded 21 sections from D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_sec_sections.parquet
INFO:rag_modules_src.entity_adapter.section_extractor:S

EmbeddingRuntimeConfig loaded


INFO:rag_modules_src.utilities.query_embedder_v2:[QueryEmbedderV2] Initialized with model=cohere.embed-v4:0, region=us-east-1, dim=1024


✓ All components initialized


In [2]:
from pathlib import Path
import logging

from loaders.ml_config_loader import MLConfig
from rag_modules_src.entity_adapter.entity_adapter import EntityAdapter
from rag_modules_src.utilities.query_embedder_v2 import (
    EmbeddingRuntimeConfig,
    QueryEmbedderV2,
    QueryTooLongError,
    QueryTooShortError,
    QueryOutOfScopeError,
)

logging.basicConfig(level=logging.INFO)

# 1) Service object: load config + credentials
config = MLConfig()  # this loads ml_config.yaml + aws_credentials.env

print("✓ MLConfig loaded")
print("  Bucket:", config.bucket)
print("  Region:", config.region)
print("  Bedrock model:", config.bedrock_model_id)

# 2) Build runtime embedding config from the same YAML embedding section
embedding_cfg_dict = config.cfg["embedding"]
runtime_cfg = EmbeddingRuntimeConfig.from_ml_config(embedding_cfg_dict)
runtime_cfg.max_query_chars = 1500  # guardrail

print("\nEmbeddingRuntimeConfig:")
print(" ", runtime_cfg)

# 3) Bedrock client from service object
bedrock_client = config.get_bedrock_client()
print("\n✓ Bedrock client created via MLConfig.get_bedrock_client()")

# 4) Initialize upstream components
adapter = EntityAdapter()
embedder = QueryEmbedderV2(runtime_cfg, boto_client=bedrock_client)

print("\n✓ EntityAdapter initialized")
print("✓ QueryEmbedderV2 initialized (using service client)")


[DEBUG] ✓ AWS credentials loaded from aws_credentials.env

INFO:rag_modules_src.entity_adapter.company_universe:Loading company dim from: D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_companies_21.parquet
INFO:rag_modules_src.entity_adapter.company_universe:Loaded dim with 21 rows and columns: ['company_id', 'cik_int', 'cik', 'company_name', 'ticker', 'source', 'tier', 'quality_score', 'selection_source', 'rank_within_group', 'selected_at', 'version']


INFO:rag_modules_src.entity_adapter.company_universe:Building indexes from dim with 21 valid rows
INFO:rag_modules_src.entity_adapter.company_universe:CompanyUniverse initialized: 21 companies, 21 tickers, 21 alias tokens
INFO:rag_modules_src.entity_adapter.section_universe:SectionUniverse loaded 21 sections from D:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\data_cache\dimensions\finrag_dim_sec_sections.parquet
INFO:rag_modules_src.entity_adapter.section_extractor:SectionExtractor initialized with 75 keyword rules, 22 item patterns, 7 risk topics
INFO:rag_modules_src.entity_adapter.metric_adapter:FilterExtractorV2 initialized with 94 metric mapping entries from metric_mapping_v2
INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter initialized successfully
INFO:rag_modules_src.utilities.query_embedder_v2:[QueryEmbedderV2] Initialized with model=cohere.embed-v4:0, region=us-east-1, dim=1024



✓ MLConfig loaded
  Bucket: sentence-data-ingestion
  Region: us-east-1
  Bedrock model: cohere.embed-v4:0

EmbeddingRuntimeConfig:
  EmbeddingRuntimeConfig(provider='bedrock', region='us-east-1', model_id='cohere.embed-v4:0', dimensions=1024, input_type='search_document', max_query_chars=1500)

✓ Bedrock client created via MLConfig.get_bedrock_client()

✓ EntityAdapter initialized
✓ QueryEmbedderV2 initialized (using service client)


In [4]:

query = "What was NVIDIA's revenue and net income in 2021 and 2022?"

# Step 1–2: entity extraction
entities = adapter.extract(query)

print("=== EntityExtractionResult (debug) ===")
try:
    print(entities.to_dict())
except AttributeError:
    print(vars(entities))

# Step 3: query embedding via Bedrock (small real cost)
embedding = embedder.embed_query(query, entities)

print("\n=== Embedding Info ===")
print("Dimension:", len(embedding))
print("First 8 dims:", embedding[:8])
print("Last 8 dims:", embedding[-8:])




INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: starting for query="What was NVIDIA's revenue and net income in 2021 and 2022?"
INFO:rag_modules_src.entity_adapter.company_extractor:Extracting companies from query: "What was NVIDIA's revenue and net income in 2021 and 2022?"
INFO:rag_modules_src.entity_adapter.company_extractor:Extraction result: ciks_int=[1045810], tickers=['NVDA'], names=['NVIDIA CORP']
INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter extracting metrics from query: "What was NVIDIA's revenue and net income in 2021 and 2022?"
INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter found metrics: ['income_stmt_Net Income', 'income_stmt_Revenue']
INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: done. companies=1, years=2, metrics=2, sections=0, risk_topics=0


=== EntityExtractionResult (debug) ===
{'query': "What was NVIDIA's revenue and net income in 2021 and 2022?", 'companies': {'ciks_int': [1045810], 'ciks_str': ['0001045810'], 'tickers': ['NVDA'], 'names': ['NVIDIA CORP']}, 'years': {'years': [2021, 2022], 'past_years': [2021, 2022], 'current_years': [], 'future_years': [], 'warning': None}, 'metrics': {'metrics': ['income_stmt_Net Income', 'income_stmt_Revenue']}, 'sections': {'items': [], 'primary': None}, 'risk_topics': []}

=== Embedding Info ===
Dimension: 1024
First 8 dims: [-0.0138549805, -0.018798828, 0.001159668, 0.028076172, 0.014526367, -0.012634277, 0.03491211, -0.016235352]
Last 8 dims: [0.0016555786, 0.03881836, -0.031982422, 0.030639648, -0.083984375, 0.024658203, 0.0013275146, 0.0154418945]


In [5]:
# 1) Too-long query → QueryTooLongError
long_query = "A" * 2001  # > max_query_chars

try:
    long_entities = adapter.extract(long_query)
    embedder.embed_query(long_query, long_entities)
    print("UNEXPECTED: long query passed guardrail.")
except QueryTooLongError as e:
    print("[OK] Caught QueryTooLongError for long query:")
    print(" ", e)
except Exception as e:
    print("[Unexpected exception type]", type(e), e)


# 2) Extremely short + likely no entities → TooShort or OutOfScope
short_query = "hi"

try:
    short_entities = adapter.extract(short_query)
    embedder.embed_query(short_query, short_entities)
    print("UNEXPECTED: short query passed guardrail.")
except QueryTooShortError as e:
    print("[OK] Caught QueryTooShortError for short query:")
    print(" ", e)
except QueryOutOfScopeError as e:
    # Depending on how EntityAdapter behaves, might go straight to out-of-scope
    print("[OK] Caught QueryOutOfScopeError for short query:")
    print(" ", e)
except Exception as e:
    print("[Unexpected exception type]", type(e), e)


# 3) Normal-length but clearly non-financial → OutOfScope
oos_query = "What is the weather today in Boston?"

try:
    oos_entities = adapter.extract(oos_query)
    embedder.embed_query(oos_query, oos_entities)
    print("UNEXPECTED: out-of-scope query passed guardrail.")
except QueryOutOfScopeError as e:
    print("[OK] Caught QueryOutOfScopeError for non-financial query:")
    print(" ", e)
except Exception as e:
    print("[Unexpected exception type]", type(e), e)


INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: starting for query='AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

[OK] Caught QueryTooLongError for long query:
  Query has 2001 characters; limit is 1500.
[OK] Caught QueryTooShortError for short query:
  Query too short and no entities detected.
[OK] Caught QueryOutOfScopeError for non-financial query:
  Query does not reference any financial/SEC concepts.


In [6]:
## Non empty and Non-bad queries // Multiple “good” queries with different shapes

test_queries = [
    "What was NVIDIA's revenue and net income in 2021 and 2022?",
    "Alphabet's liquidity risk trends in Item 7A for 2019.",
    "How did Apple’s operating cash flow evolve between 2015 and 2020?",
    "Regulatory and legal risks for Tesla in 2018 in Item 1A.",
]

for q in test_queries:
    print("\n" + "=" * 80)
    print("QUERY:", q)

    # Entity extraction
    ents = adapter.extract(q)
    try:
        ents_dict = ents.to_dict()
    except AttributeError:
        ents_dict = vars(ents)

    print("Entities summary:")
    print("  companies:", ents_dict.get("companies"))
    print("  years    :", ents_dict.get("years"))
    print("  metrics  :", ents_dict.get("metrics"))
    print("  sections :", ents_dict.get("sections"))
    print("  risks    :", ents_dict.get("risk_topics"))

    # Embedding
    try:
        emb = embedder.embed_query(q, ents)
        print("Embedding dim:", len(emb))
        print("First 4 dims:", emb[:4])
        print("Last 4 dims :", emb[-4:])
    except Exception as e:
        print("ERROR during embedding:", type(e).__name__, "-", e)


INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: starting for query="What was NVIDIA's revenue and net income in 2021 and 2022?"
INFO:rag_modules_src.entity_adapter.company_extractor:Extracting companies from query: "What was NVIDIA's revenue and net income in 2021 and 2022?"
INFO:rag_modules_src.entity_adapter.company_extractor:Extraction result: ciks_int=[1045810], tickers=['NVDA'], names=['NVIDIA CORP']
INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter extracting metrics from query: "What was NVIDIA's revenue and net income in 2021 and 2022?"
INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter found metrics: ['income_stmt_Net Income', 'income_stmt_Revenue']
INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: done. companies=1, years=2, metrics=2, sections=0, risk_topics=0
INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: starting for query="Alphabet's liquidity risk trends in Item 7A f


QUERY: What was NVIDIA's revenue and net income in 2021 and 2022?
Entities summary:
  companies: {'ciks_int': [1045810], 'ciks_str': ['0001045810'], 'tickers': ['NVDA'], 'names': ['NVIDIA CORP']}
  years    : {'years': [2021, 2022], 'past_years': [2021, 2022], 'current_years': [], 'future_years': [], 'warning': None}
  metrics  : {'metrics': ['income_stmt_Net Income', 'income_stmt_Revenue']}
  sections : {'items': [], 'primary': None}
  risks    : []
Embedding dim: 1024
First 4 dims: [-0.0138549805, -0.018798828, 0.001159668, 0.028076172]
Last 4 dims : [-0.083984375, 0.024658203, 0.0013275146, 0.0154418945]

QUERY: Alphabet's liquidity risk trends in Item 7A for 2019.


INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter found metrics: []
INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: done. companies=1, years=1, metrics=0, sections=3, risk_topics=1
INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: starting for query='How did Apple’s operating cash flow evolve between 2015 and 2020?'
INFO:rag_modules_src.entity_adapter.company_extractor:Extracting companies from query: 'How did Apple’s operating cash flow evolve between 2015 and 2020?'
INFO:rag_modules_src.entity_adapter.company_extractor:Extraction result: ciks_int=[320193], tickers=['AAPL'], names=['Apple Inc.']
INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter extracting metrics from query: 'How did Apple’s operating cash flow evolve between 2015 and 2020?'
INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter found metrics: ['cash_flow_Operating Cash Flow']
INFO:rag_modules_src.entity_adapter.entity_adapter:En

Entities summary:
  companies: {'ciks_int': [1652044], 'ciks_str': ['0001652044'], 'tickers': ['GOOGL'], 'names': ['Alphabet Inc.']}
  years    : {'years': [2019], 'past_years': [2019], 'current_years': [], 'future_years': [], 'warning': None}
  metrics  : {'metrics': []}
  sections : {'items': ['ITEM_7A', 'ITEM_7', 'ITEM_1A'], 'primary': 'ITEM_7'}
  risks    : ['liquidity_credit']
Embedding dim: 1024
First 4 dims: [0.060058594, -0.05810547, -0.02722168, 0.013977051]
Last 4 dims : [-0.011474609, 0.07861328, 0.0011444092, 0.04345703]

QUERY: How did Apple’s operating cash flow evolve between 2015 and 2020?
Entities summary:
  companies: {'ciks_int': [320193], 'ciks_str': ['0000320193'], 'tickers': ['AAPL'], 'names': ['Apple Inc.']}
  years    : {'years': [2015, 2020], 'past_years': [2015, 2020], 'current_years': [], 'future_years': [], 'warning': None}
  metrics  : {'metrics': ['cash_flow_Operating Cash Flow']}
  sections : {'items': ['ITEM_1A'], 'primary': 'ITEM_1A'}
  risks    : ['liq

INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: starting for query='Regulatory and legal risks for Tesla in 2018 in Item 1A.'
INFO:rag_modules_src.entity_adapter.company_extractor:Extracting companies from query: 'Regulatory and legal risks for Tesla in 2018 in Item 1A.'
INFO:rag_modules_src.entity_adapter.company_extractor:Extraction result: ciks_int=[1318605], tickers=['TSLA'], names=['Tesla, Inc.']
INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter extracting metrics from query: 'Regulatory and legal risks for Tesla in 2018 in Item 1A.'
INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter found metrics: []
INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: done. companies=1, years=1, metrics=0, sections=1, risk_topics=1


Embedding dim: 1024
First 4 dims: [-0.041259766, -0.013671875, 0.01953125, 0.012329102]
Last 4 dims : [0.04296875, 0.040771484, 0.012512207, 0.020385742]

QUERY: Regulatory and legal risks for Tesla in 2018 in Item 1A.
Entities summary:
  companies: {'ciks_int': [1318605], 'ciks_str': ['0001318605'], 'tickers': ['TSLA'], 'names': ['Tesla, Inc.']}
  years    : {'years': [2018], 'past_years': [2018], 'current_years': [], 'future_years': [], 'warning': None}
  metrics  : {'metrics': []}
  sections : {'items': ['ITEM_1A'], 'primary': 'ITEM_1A'}
  risks    : ['regulatory']
Embedding dim: 1024
First 4 dims: [-0.03515625, -0.017944336, -0.006500244, -0.0035858154]
Last 4 dims : [0.016235352, 0.057373047, 0.048095703, 0.034179688]


In [7]:
## Systematic guardrail outcomes table

from typing import Type, Optional

tests = [
    ("too long", "A" * 2001, QueryTooLongError),
    ("too short", "hi", (QueryTooShortError, QueryOutOfScopeError)),  # either acceptable
    ("weather", "What is the weather today in Boston?", QueryOutOfScopeError),
    ("bare company", "Apple Inc", None),  # should be allowed as in-scope
    ("metric only", "Show me revenue and net income.", None),  # should be allowed if metrics detected
]

def expected_match(exc: Exception, expected_type: Optional[Type[Exception] | tuple]) -> bool:
    if expected_type is None:
        return False
    if isinstance(expected_type, tuple):
        return any(isinstance(exc, t) for t in expected_type)
    return isinstance(exc, expected_type)

for name, q, expected_exc in tests:
    print("\n" + "-" * 80)
    print(f"Test case: {name}")
    print("Query:", q)

    ents = adapter.extract(q)

    try:
        emb = embedder.embed_query(q, ents)
        if expected_exc is None:
            print("Result: PASSED guardrails (expected). Embedding dim:", len(emb))
        else:
            print("Result: UNEXPECTEDLY passed, expected exception:", expected_exc)
    except Exception as e:
        if expected_match(e, expected_exc):
            print(f"Result: Correctly raised {type(e).__name__}: {e}")
        else:
            print(f"Result: UNEXPECTED exception {type(e).__name__}: {e}")


INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: starting for query='AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


--------------------------------------------------------------------------------
Test case: too long
Query: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter found metrics: []
INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: done. companies=0, years=0, metrics=0, sections=0, risk_topics=0
INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: starting for query='hi'
INFO:rag_modules_src.entity_adapter.company_extractor:Extracting companies from query: 'hi'
INFO:rag_modules_src.entity_adapter.company_extractor:Extraction result: ciks_int=[], tickers=[], names=[]
INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter extracting metrics from query: 'hi'
INFO:rag_modules_src.entity_adapter.metric_adapter:MetricAdapter found metrics: []
INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: done. companies=0, years=0, metrics=0, sections=0, risk_topics=0
INFO:rag_modules_src.entity_adapter.entity_adapter:EntityAdapter.extract: starting for query='What is the weather today in Boston?'
INFO:rag_modules_src.entit

Result: Correctly raised QueryTooLongError: Query has 2001 characters; limit is 1500.

--------------------------------------------------------------------------------
Test case: too short
Query: hi
Result: Correctly raised QueryTooShortError: Query too short and no entities detected.

--------------------------------------------------------------------------------
Test case: weather
Query: What is the weather today in Boston?
Result: Correctly raised QueryOutOfScopeError: Query does not reference any financial/SEC concepts.

--------------------------------------------------------------------------------
Test case: bare company
Query: Apple Inc
Result: PASSED guardrails (expected). Embedding dim: 1024

--------------------------------------------------------------------------------
Test case: metric only
Query: Show me revenue and net income.
Result: PASSED guardrails (expected). Embedding dim: 1024


In [ ]:
# from pathlib import Path
# import sys

# def find_project_root(marker_names=None, start_path=None):
#     """
#     Walk UP the directory tree until finding a folder containing ANY of the marker items.
    
#     Args:
#         marker_names: List of files/folders that identify project root
#         start_path: Where to start searching (defaults to current working directory)
    
#     Returns:
#         Path object of project root
    
#     Raises:
#         RuntimeError if project root not found
#     """
#     if marker_names is None:
#         marker_names = ['rag_modules_src', '.git', 'pyproject.toml', 'setup.py']
    
#     current = Path(start_path) if start_path else Path.cwd()
    
#     for parent in [current] + list(current.parents):
#         # Check if ANY marker exists in this directory
#         if any((parent / marker).exists() for marker in marker_names):
#             return parent
    
#     raise RuntimeError(
#         f"Could not find project root. Searched for markers: {marker_names}\n"
#         f"Started from: {current}"
#     )

# project_root = find_project_root(marker_names=['rag_modules_src', '.git'])
# if str(project_root) not in sys.path:
#     sys.path.insert(0, str(project_root))

# print(f"✓ Project root: {project_root}")
# print(f"✓ sys.path updated")

# import rag_modules_src
# from rag_modules_src.entity_adapter import company_extractor